# Collaborative Filtering

In [1]:
# data is from https://www.kaggle.com/CooperUnion/anime-recommendations-database

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD

import warnings; warnings.simplefilter('ignore')



from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
reader = Reader()
ratings = pd.read_csv('./input/rating.csv')
ratings = ratings.dropna()
ratings.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [2]:
ratings["anime_id"].nunique()

11200

In [3]:
data = Dataset.load_from_df(ratings[['user_id', 'anime_id', 'rating']], reader)
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'])

{'test_rmse': array([3.28955846, 3.29040951, 3.29113491, 3.29079773, 3.2902267 ]),
 'test_mae': array([2.95035964, 2.95168714, 2.95189535, 2.95212547, 2.95110548]),
 'fit_time': (291.02139258384705,
  300.0403804779053,
  294.4403119087219,
  304.7607340812683,
  309.06429147720337),
 'test_time': (17.916151762008667,
  19.285151720046997,
  18.354126930236816,
  18.301060676574707,
  18.295573234558105)}

In [4]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [5]:
svd.predict(1, 9, 1)

Prediction(uid=1, iid=9, r_ui=1, est=2.3843303947105943, details={'was_impossible': False})

In [6]:
ratings[ratings['user_id'] == 1]

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1
...,...,...,...
148,1,30015,-1
149,1,30296,-1
150,1,30544,-1
151,1,31338,-1


# Hybrid-content base +Collaborative Filtering

In [45]:
data1 = pd.read_csv("./input/anime.csv")
data1 = data1.rename({'rating':'AverageRating'}, axis='columns')
anime_id_ratings=set([i for i in ratings.anime_id])     
         
data1 = data1[data1['anime_id'].isin(anime_id_ratings)]

In [26]:
data1.head(2)

,anime_id,name,genre,type,episodes,AverageRating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665


In [27]:
data1 = data1.dropna()
data1.isnull().sum()

anime_id         0
name             0
genre            0
type             0
episodes         0
AverageRating    0
members          0
dtype: int64

In [28]:
print(data1["anime_id"].nunique())
data1['genre'] = data1["genre"].apply(lambda x: x.split(","))
data1['genre'] = data1['genre'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])


11162


In [29]:
data1["soup"] = data1["genre"]+data1["type"].map(lambda x : [x])
data1['soup'] = data1['soup'].apply(lambda x: ' '.join(x))

count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(data1["soup"])
cosine_sim = cosine_similarity(count_matrix, count_matrix)


In [30]:
indices = pd.Series(data1.index, index=data1['name'])
indices.head(1)

name
Kimi no Na wa.    0
dtype: int64

In [31]:
id_map = pd.read_csv('./input/anime.csv')[["anime_id", "name"]].set_index('name')
id_map.head(1)

,anime_id
name,
Kimi no Na wa.,32281


In [38]:
def hybrid(userId, title):
    idx = indices[title]

    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    anime_indices = [i[0] for i in sim_scores]
    
    
    anime = data1.iloc[anime_indices][["name",'genre', 'AverageRating', 'anime_id', "type"]]
    anime['est'] = anime['anime_id'].apply(lambda x: svd.predict(userId, x).est)
    
    anime = anime.sort_values('est', ascending=False)
    return anime.head(10)

In [39]:
hybrid(50, 'Kimi no Na wa.')

,name,genre,AverageRating,anime_id,type,est
2300,Koi to Senkyo to Chocolate,"[drama, romance, school]",7.30,12175,TV,5.0
3297,Koi to Senkyo to Chocolate Special,"[drama, romance, school]",7.03,18045,Special,5.0
1697,Zutto Mae kara Suki deshita.: Kokuhaku Jikkou ...,"[romance, school]",7.47,31245,Movie,5.0
894,Momo e no Tegami,"[drama, supernatural]",7.78,10389,Movie,5.0
3544,Ace wo Nerae! (1979),"[drama, romance, school, shoujo, sports]",6.95,313,Movie,5.0
2060,Takanashi Rikka Kai: Chuunibyou demo Koi ga Sh...,"[comedy, drama, romance, school, sliceoflife]",7.37,19021,Movie,5.0
6160,Tokimeki Memorial: Forever With You,"[drama, romance, school]",6.24,2179,OVA,5.0
6156,School Days ONA,"[drama, romance, school]",6.24,756,ONA,5.0
5233,To Heart 2 Special,"[drama, romance, school]",6.52,1624,Special,5.0
3914,Myself ; Yourself Specials,"[drama, romance, school]",6.85,17585,Special,5.0


In [40]:
hybrid(5, 'Kimi no Na wa.')

,name,genre,AverageRating,anime_id,type,est
1111,Aura: Maryuuin Kouga Saigo no Tatakai,"[comedy, drama, romance, school, supernatural]",7.67,14669,Movie,5.000000
1436,&quot;Bungaku Shoujo&quot; Memoire,"[drama, romance, school]",7.54,8481,OVA,5.000000
894,Momo e no Tegami,"[drama, supernatural]",7.78,10389,Movie,5.000000
208,Kokoro ga Sakebitagatterunda.,"[drama, romance, school]",8.32,28725,Movie,5.000000
3544,Ace wo Nerae! (1979),"[drama, romance, school, shoujo, sports]",6.95,313,Movie,5.000000
1959,Air Movie,"[drama, romance, supernatural]",7.39,713,Movie,5.000000
60,Hotarubi no Mori e,"[drama, romance, shoujo, supernatural]",8.61,10408,Movie,5.000000
2060,Takanashi Rikka Kai: Chuunibyou demo Koi ga Sh...,"[comedy, drama, romance, school, sliceoflife]",7.37,19021,Movie,4.938089
6119,Shisha no Sho,"[drama, supernatural]",6.25,7510,Movie,4.624851
1494,Harmonie,"[drama, school, supernatural]",7.52,20903,Movie,4.571846


In [42]:
hybrid(5, 'Death Note')

,name,genre,AverageRating,anime_id,type,est
8984,Ishii Hisaichi no Dai Seikai,"[comedy, parody]",5.13,19583,OVA,5.000000
9545,Mikosuri Han-Gekijou,"[comedy, parody]",5.43,20453,OVA,5.000000
4775,Mobile Suit SD Gundam&#039;s Counterattack,"[comedy, mecha, parody]",6.64,2306,OVA,5.000000
5469,Scramble Wars: Tsuppashire! Genom Trophy Rally,[parody],6.45,2466,OVA,5.000000
512,Carnival Phantasm,"[comedy, parody, supernatural]",8.01,10012,OVA,5.000000
575,Goku Sayonara Zetsubou Sensei,"[comedy, parody, school]",7.96,4872,OVA,5.000000
618,Zan Sayonara Zetsubou Sensei Bangaichi,"[comedy, parody, school]",7.94,7044,OVA,5.000000
2364,Genshiken Nidaime OVA,"[comedy, parody, sliceoflife]",7.28,19159,OVA,5.000000
8880,Himitsukessha Taka no Tsume The Movie 4: Kaspe...,"[comedy, parody]",6.40,30753,OVA,4.645604
9946,Peeping Life: Tezuka Pro - Tatsunoko Pro Wonde...,"[comedy, parody]",4.88,22631,OVA,4.522167


In [43]:
hybrid(7, 'Sword Art Online')

,name,genre,AverageRating,anime_id,type,est
717,Shingeki no Kyojin OVA,"[action, drama, fantasy, shounen, superpower]",7.88,18397,OVA,5
146,Hunter x Hunter: Greed Island Final,"[action, adventure, shounen, superpower]",8.41,139,OVA,5
1069,Shingeki no Kyojin Movie 1: Guren no Yumiya,"[action, drama, fantasy, shounen, superpower]",7.70,23775,Movie,5
961,Shingeki no Kyojin Movie 2: Jiyuu no Tsubasa,"[action, drama, fantasy, shounen, superpower]",7.75,23777,Movie,5
86,Shingeki no Kyojin,"[action, drama, fantasy, shounen, superpower]",8.54,16498,TV,5
7284,Shinrabanshou: Tenchi Shinmei no Shou,"[action, fantasy, shounen]",5.58,24719,ONA,5
3437,Juushin Enbu: Hero Tales,"[action, fantasy, shounen]",6.99,2772,TV,5
4071,Kekkai Sensen: Soresaemo Saitei de Saikou na Hibi,"[action, fantasy, shounen, superpower, superna...",6.80,30915,Special,5
978,Kekkai Sensen,"[action, fantasy, shounen, superpower, superna...",7.74,24439,TV,5
2617,Munto: Toki no Kabe wo Koete,"[action, fantasy, magic, superpower]",7.21,1135,OVA,5


In [44]:
hybrid(20, 'Shingeki no Kyojin')

,name,genre,AverageRating,anime_id,type,est
294,Evangelion: 1.0 You Are (Not) Alone,"[action, mecha, sci-fi]",8.21,2759,Movie,5
10083,Robot Taekwon V 3tan! Sujung Teukgongdae,"[action, mecha, sci-fi]",4.81,16566,Movie,5
8424,Dallyeola Majing-ga-X,"[mecha, sci-fi]",4.13,16554,Movie,5
7683,Super Express Mazinger 7,"[mecha, sci-fi]",5.15,3252,Movie,5
6524,Wakusei Robo Danguard Ace tai Konchuu Robot Gu...,"[mecha, sci-fi]",6.08,8707,Movie,5
6332,Wakusei Robo Danguard Ace: Uchuu Daikaisen,"[mecha, sci-fi]",6.17,5515,Movie,5
6018,Tetsuwan Atom: Kagayakeru Hoshi - Anata wa Aok...,"[mecha, sci-fi]",6.28,17963,Movie,5
4704,UFO Robo Grendizer tai Great Mazinger,"[mecha, sci-fi]",6.66,4387,Movie,5
4541,UFO Robo Grendizer: Akai Yuuhi no Taiketsu,"[mecha, sci-fi]",6.70,7839,Movie,5
10451,Syupeo Taegwon V,"[action, mecha, sci-fi]",4.10,16568,Movie,5
